Reference for joints:

<img src = "https://i.imgur.com/3j8BPdc.png" style = "height : 300px">

Installations & import:

In [14]:
!pip install mediapipe opencv-python

import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [8]:
def resize_frame(frame):
    # Define the maximum width and height
    max_width = 800  # Adjust this according to your preference
    max_height = 600  # Adjust this according to your preference

    # Get the current width and height of the frame
    height, width, _ = frame.shape

    # Calculate the aspect ratio
    aspect_ratio = width / height

    # Calculate new dimensions while maintaining aspect ratio
    if width > height:
        new_width = min(width, max_width)
        new_height = int(new_width / aspect_ratio)
    else:
        new_height = min(height, max_height)
        new_width = int(new_height * aspect_ratio)

    # Resize the frame
    resized_frame = cv2.resize(frame, (new_width, new_height))
    return resized_frame

In [10]:
min_angle_elbow = 25
max_angle_elbow = 60

min_angle_armpit = 110
max_angle_armpit = 140

def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [13]:
# VIDEO FILE
#wrong position video::
#video_file = r"C:\FilesBeth15022024\ludus project dataset\dataset_warriorSlap_ludus-20240330T195138Z-001\dataset_warriorSlap_ludus\warrior_slap_videos\VID_20240315_154800.mp4"

#right position video:
#video_file = r"C:\FilesBeth15022024\ludus project dataset\dataset_warriorSlap_ludus-20240330T195138Z-001\dataset_warriorSlap_ludus\warrior_slap_videos\VID_20240315_154836.mp4"

#video_file = r"C:\FilesBeth15022024\ludus project dataset\dataset_warriorSlap_ludus-20240330T195138Z-001\dataset_warriorSlap_ludus\warrior_slap_videos\VID_20240315_155139.mp4"

video_file = r"C:\FilesBeth15022024\ludus project dataset\dataset_warriorSlap_ludus-20240330T195138Z-001\dataset_warriorSlap_ludus\warrior_slap_videos\VID_20240315_154735.mp4"

cap = cv2.VideoCapture(video_file)

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        resized_frame = resize_frame(frame)
        # Recolor image to RGB
        image = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
                
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates for left:
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_waist = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            # Get coordinates for right:
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            right_waist = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            # Calculate angles for both left and right:
            right_elbow_angle = calculate_angle(right_wrist, right_elbow, right_shoulder)
            right_armpit_angle = calculate_angle(right_elbow, right_shoulder, right_waist)

            left_elbow_angle = calculate_angle(left_wrist, left_elbow, left_shoulder)
            left_armpit_angle = calculate_angle(left_elbow, left_shoulder, left_waist)
            
            if min_angle_elbow <= right_elbow_angle <= max_angle_elbow and min_angle_armpit <= right_armpit_angle <= max_angle_armpit:
                text_color = (255, 255, 255)  # White color
            else:
                text_color = (0, 0, 255)  # Red color

            # Visualizing the angles: 
            cv2.putText(image, "Right Elbow: {}".format(int(right_elbow_angle)), 
                       tuple(np.multiply(right_elbow, [350, 600]).astype(int)), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.2, text_color, 1, cv2.LINE_AA)

            cv2.putText(image, "Right Armpit: {}".format(int(right_armpit_angle)), 
                       tuple(np.multiply(right_shoulder, [350, 600]).astype(int)), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.2, text_color, 1, cv2.LINE_AA)

            cv2.putText(image, "Left Elbow: {}".format(int(left_elbow_angle)), 
                       tuple(np.multiply(left_elbow, [100, 600]).astype(int)), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.2, text_color, 1, cv2.LINE_AA)

            cv2.putText(image, "Left Armpit: {}".format(int(left_armpit_angle)), 
                       tuple(np.multiply(left_shoulder, [100, 600]).astype(int)), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.2, text_color, 1, cv2.LINE_AA)

        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()